In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from functools import partial
import cv2 
from glob import glob
import matplotlib.pyplot as plt
import scipy.io as io

In [2]:
def get_width(arr, direction): # dlib's eye 6 point landamrk 
    if direction == 0:  # left
        width_top = [(arr[2] + arr[4]) // 2, (arr[3] + arr[5]) // 2]
        width_bottom = [(arr[8] + arr[10]) // 2, (arr[9] + arr[11]) // 2]
    if direction == 1: # right 
        width_top = [(arr[14] + arr[16]) // 2, (arr[15] + arr[17]) // 2]
        width_bottom = [(arr[20] + arr[22]) // 2, (arr[21] + arr[23]) // 2]
    return width_top, width_bottom

def get_height(arr, direction):
    if direction == 0:  # left
        height_left = [arr[0], arr[1]]
        height_right = [arr[6], arr[7]]
    if direction == 1: # right 
        height_left = [arr[12], arr[13]]
        height_right = [arr[18], arr[19]]
    return height_left, height_right

def draw_dot(img_path, arr):
    image = cv2.imread(img_path)
    image_dot = cv2.line(image,(arr[0],arr[1]),(arr[0],arr[1]),(255,0,0),5)
    plt.imshow(image_dot)

In [3]:
def make_coordinate(arr, direction):
    if direction == 0:
        top, bottom = get_width(arr, 0)
        left, right = get_height(arr, 0)
    elif direction == 1:
        top, bottom = get_width(arr, 1)
        left, right = get_height(arr, 1)

In [4]:
def add(array,mod, num = 15):
    arr = array.copy()
    if mod == 0: #left
        arr[0] -= num
    elif mod == 1: #right
        arr[0] += num
    elif mod == 2: # top
        arr[1] -= num
    elif mod == 3: # bottom
        arr[1] += num
    return arr

In [11]:
def Normalize(arr,img_path, output_w = 120, output_h = 80):
    mod_left = add(arr[0], 0)
    mod_right = add(arr[1], 1)
    mod_top = add(arr[2], 2)
    mod_bottom = add(arr[3], 3)
    
    rect_left_top = [mod_left[0], mod_top[1]]
    rect_right_top = [mod_right[0], mod_top[1]]
    rect_left_bottom = [mod_left[0], mod_bottom[1]]
    rect_right_bottom = [mod_right[0], mod_bottom[1]]
    
    img = cv2.imread(img_path)
    h, w, c = img.shape
    
    dstPoint=np.array([rect_left_top, rect_right_top, rect_right_bottom,
                   rect_left_bottom], dtype=np.float32)
    srcPoint=np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(dstPoint, srcPoint)
    
    dst = cv2.warpPerspective(img, matrix, (w, h))
    resized_dst = cv2.resize(dst, (output_w, output_h), interpolation=cv2.INTER_AREA )
    return resized_dst

In [6]:
image_path_divide = glob('/home/spow12/Codes/Data/MPIIGaze/Data/Original/*/*') # 521개
image_path_divide.sort()
day_list = [39, 69, 39, 65, 25, 38, 62, 56, 47, 20, 16, 19, 7, 12, 7]  
size_li = []
for i in range(len(image_path_divide)):
    s = len(glob(image_path_divide[i] + '/*'))
    if s != 3:
        size_li.append(s-1)

day_size = []
t = []
cnt = 1
idx = 0
for size in size_li:
    t.append(size)
    if cnt == day_list[idx]:
        cnt = 0
        idx += 1
        day_size.append(t)
        t = []
    cnt += 1
print(len(day_size))
print(len(day_size[0]), len(day_size[1]))

15
39 69


In [7]:
anno_path = glob('/home/spow12/Codes/Data/MPIIGaze/Data/Original/*/*/annotation.txt')
image_path = glob('/home/spow12/Codes/Data/MPIIGaze/Data/Original/*/*/*.jpg')

#image_path = glob('MPIIGaze/Data/Original/*/*/*.jpg')
#anno_path = glob('MPIIGaze/Data/Original/*/*/annotation.txt')
image_path.sort()
anno_path.sort()
print(len(image_path), len(anno_path))

213658 521


In [8]:
import os

dir_path_right = '/home/spow12/Codes/Data/MPIIGaze/Data/Our_data_right_mod/'
dir_path_left = '/home/spow12/Codes/Data/MPIIGaze/Data/Our_data_left_mod/'

In [9]:
def for_sorting(cnt):
    cnt = str(cnt)
    if len(cnt) < 4:
        cnt = '0' * (4 - len(cnt)) + cnt
    return cnt

In [12]:
normal = []
cnt = 0
size_idx = 0
day_idx = 0
check = 0

for path in image_path:
    anno_df = pd.read_csv(anno_path[size_idx], sep = ' ' ,header= None)
    landmarks = anno_df.iloc[:, :24]
    land_ls = list(landmarks.loc[cnt])
    img = cv2.imread(path)

    top, bottom = get_width(land_ls, 1)  # 0 -> left, 1 -> right
    left, right = get_height(land_ls, 1)

    array = [left, right, top, bottom]  # Integrate value for making function's input
    dst = Normalize(array, path)
    normal.append(dst)
    
    str_day_idx = for_sorting(day_idx)
    str_check = for_sorting(check)
    
    dir_path2 = 'p' + str_day_idx
    dir_path3 = 'day' + str_check + '/'
    path = dir_path_left + "/" + dir_path2 + "/" + dir_path3
    
    try:
        if not(os.path.isdir(path)):
            os.makedirs(os.path.join(path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise
    
    cnt2 = for_sorting(cnt)
    cv2.imwrite(path + '{0}.jpg'.format(cnt2), dst)
    
    if cnt == size_li[size_idx] - 1:
        print(cnt, day_idx, size_idx, check)
        cnt = -1
        size_idx += 1
        check += 1
    
#         if check == 3:
#             break
        
    if check == day_list[day_idx]:
        day_idx += 1
        check = 0
        normal = []
        exec('p%d_noramlize = normal' % (day_idx))
    
    cnt += 1

994 0 0 0
857 0 1 1
1067 0 2 2
1239 0 3 3
519 0 4 4
739 0 5 5
277 0 6 6
757 0 7 7
999 0 8 8
896 0 9 9
532 0 10 10
519 0 11 11
759 0 12 12
479 0 13 13
579 0 14 14
1099 0 15 15
1079 0 16 16
515 0 17 17
599 0 18 18
239 0 19 19
699 0 20 20
919 0 21 21
919 0 22 22
1166 0 23 23
739 0 24 24
792 0 25 25
619 0 26 26
779 0 27 27
839 0 28 28
638 0 29 29
759 0 30 30
519 0 31 31
779 0 32 32
839 0 33 33
958 0 34 34
859 0 35 35
639 0 36 36
1299 0 37 37
419 0 38 38
509 1 39 0
230 1 40 1
576 1 41 2
667 1 42 3
66 1 43 4
415 1 44 5
339 1 45 6
184 1 46 7
69 1 47 8
217 1 48 9
201 1 49 10
587 1 50 11
196 1 51 12
315 1 52 13
527 1 53 14
379 1 54 15
569 1 55 16
82 1 56 17
200 1 57 18
596 1 58 19
315 1 59 20
249 1 60 21
268 1 61 22
11 1 62 23
235 1 63 24
59 1 64 25
531 1 65 26
299 1 66 27
74 1 67 28
85 1 68 29
455 1 69 30
240 1 70 31
112 1 71 32
299 1 72 33
193 1 73 34
176 1 74 35
537 1 75 36
513 1 76 37
0 1 77 38
39 1 78 39
532 1 79 40
716 1 80 41
616 1 81 42
201 1 82 43
672 1 83 44
16 1 84 45
345 1 85 46
265

In [ ]:
normal = []
cnt = 0
size_idx = 0
day_idx = 0
check = 0

for path in image_path:
    anno_df = pd.read_csv(anno_path[size_idx], sep = ' ' ,header= None)
    landmarks = anno_df.iloc[:, :24]
    land_ls = list(landmarks.loc[cnt])
    img = cv2.imread(path)

    top, bottom = get_width(land_ls, 0)  # 0 -> left, 1 -> right
    left, right = get_height(land_ls, 0)

    array = [left, right, top, bottom]  # Integrate value for making function's input
    dst = Normalize(array, path)
    normal.append(dst)
    
    str_day_idx = for_sorting(day_idx)
    str_check = for_sorting(check)
    
    dir_path2 = 'p' + str_day_idx
    dir_path3 = 'day' + str_check + '/'
    path = dir_path_right + "/" + dir_path2 + "/" + dir_path3
    
    try:
        if not(os.path.isdir(path)):
            os.makedirs(os.path.join(path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise
    
    cnt2 = for_sorting(cnt)
    cv2.imwrite(path + '{0}.jpg'.format(cnt2), dst)
    
    if cnt == size_li[size_idx] - 1:
        print(cnt, day_idx, size_idx, check)
        cnt = -1
        size_idx += 1
        check += 1
    
#         if check == 3:
#             break
        
    if check == day_list[day_idx]:
        day_idx += 1
        check = 0
        normal = []
        exec('p%d_noramlize = normal' % (day_idx))
    
    cnt += 1

994 0 0 0
857 0 1 1
1067 0 2 2
1239 0 3 3
519 0 4 4
739 0 5 5
277 0 6 6
757 0 7 7
999 0 8 8
896 0 9 9
532 0 10 10
519 0 11 11
759 0 12 12
479 0 13 13
579 0 14 14
1099 0 15 15
1079 0 16 16
515 0 17 17
599 0 18 18
239 0 19 19
699 0 20 20
919 0 21 21
919 0 22 22
1166 0 23 23
739 0 24 24
792 0 25 25
619 0 26 26
779 0 27 27
839 0 28 28
638 0 29 29
759 0 30 30
519 0 31 31
779 0 32 32
839 0 33 33
958 0 34 34
859 0 35 35
639 0 36 36
1299 0 37 37
419 0 38 38
509 1 39 0
230 1 40 1
576 1 41 2
667 1 42 3
66 1 43 4
415 1 44 5
339 1 45 6
184 1 46 7
69 1 47 8
217 1 48 9
201 1 49 10
587 1 50 11
196 1 51 12
315 1 52 13
527 1 53 14
379 1 54 15
569 1 55 16
82 1 56 17
200 1 57 18
596 1 58 19
315 1 59 20
249 1 60 21
268 1 61 22
11 1 62 23
235 1 63 24
59 1 64 25
531 1 65 26
299 1 66 27
74 1 67 28
85 1 68 29
455 1 69 30
240 1 70 31
112 1 71 32
299 1 72 33
193 1 73 34
176 1 74 35
537 1 75 36
513 1 76 37
0 1 77 38
39 1 78 39
532 1 79 40
716 1 80 41
616 1 81 42
201 1 82 43
672 1 83 44
16 1 84 45
345 1 85 46
265